In [1]:
import requests
import pandas as pd
import json

### Recieve DATA

In [2]:
x = requests.get('http://127.0.0.1:5000/transactions') 

In [3]:
data = x.json()

In [4]:
data.keys()

dict_keys(['error', 'transactions'])

### Pell off

In [5]:
t = data['transactions']

In [6]:
t.keys()

dict_keys(['accounts', 'item', 'request_id', 'total_transactions', 'transactions'])

In [7]:
transactions = pd.DataFrame(t['transactions'])

In [8]:
transactions = transactions[['name', 'amount', 'category', 'date', 'account_id']]


In [9]:
accounts = pd.DataFrame(t['accounts'])

In [10]:
balances = pd.DataFrame(accounts['balances'].tolist())

In [11]:
result = accounts.join(balances)

In [12]:
result = transactions.merge(result, on = 'account_id', how = 'left').reset_index()

In [13]:
result = result[['name_x', 'amount', 'category', 'date', 'account_id', 'available', 'current', 'limit']].sort_values(['account_id'], 
                                   ascending = False).reset_index().drop(['index'], axis = 1).fillna(0).rename({'name_x':'name'}, axis=1)

### Dirty data

In [14]:
result = result.fillna(0)

In [15]:
dirty = result

### Category devider

In [16]:
cat = []
for elem in result['category']: 
    res_str = ' / '.join(elem)
    cat.append(res_str)

In [17]:
result['category'] = cat

In [18]:
dirty.head()

,name,amount,category,date,account_id,available,current,limit
0,CD DEPOSIT .INITIAL.,1000.0,Transfer / Deposit,2020-05-14,xGgpoR9yn5Uo3K5m93pKTlXpDEPkBACnqqxDo,0.0,1000.0,0.0
1,Tectra Inc,500.0,Food and Drink / Restaurants,2020-05-25,dkDqKNjB68tGDQbvmDaQHKgpP6Z4AXcZmmv4k,0.0,410.0,2000.0
2,AUTOMATIC PAYMENT - THANK,2078.5,Payment,2020-05-24,dkDqKNjB68tGDQbvmDaQHKgpP6Z4AXcZmmv4k,0.0,410.0,2000.0
3,KFC,500.0,Food and Drink / Restaurants / Fast Food,2020-05-24,dkDqKNjB68tGDQbvmDaQHKgpP6Z4AXcZmmv4k,0.0,410.0,2000.0
4,Madison Bicycle Shop,500.0,Shops / Sporting Goods,2020-05-24,dkDqKNjB68tGDQbvmDaQHKgpP6Z4AXcZmmv4k,0.0,410.0,2000.0


In [19]:
import httplib2 
import apiclient.discovery
from oauth2client.service_account import ServiceAccountCredentials

### Connection to google 

In [20]:
CREDENTIALS_FILE = 'nice-karma-278512-2aa00c554b3a.json'  # Add json

credentials = ServiceAccountCredentials.from_json_keyfile_name(CREDENTIALS_FILE, ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive'])

httpAuth = credentials.authorize(httplib2.Http()) # Authorization
service = apiclient.discovery.build('sheets', 'v4', http = httpAuth)  

spreadsheet = service.spreadsheets().create(body = {
    'properties': {'title': 'DataFrame', 'locale': 'ru_RU'},
    'sheets': [{'properties': {'sheetType': 'GRID',
                               'sheetId': 0,
                               'title': 'Dirty_data',
                               'gridProperties': {'rowCount': 100, 'columnCount': 15}}}]
}).execute()
spreadsheetId = spreadsheet['spreadsheetId'] # add sheet id
print('https://docs.google.com/spreadsheets/d/' + spreadsheetId)

https://docs.google.com/spreadsheets/d/1cbkFqsdsJbmB6CPMgCJ-7q8Qz_JnvoNdyR8x3uINjt0


In [21]:
driveService = apiclient.discovery.build('drive', 'v3', http = httpAuth) 
access = driveService.permissions().create(
    fileId = spreadsheetId,
    body = {'type': 'user', 'role': 'writer', 'emailAddress': 'binancebender9@gmail.com'},  # Add service email
).execute()

In [22]:
#Add list
results = service.spreadsheets().batchUpdate(
    spreadsheetId = spreadsheetId,
    body = 
{
  "requests": [
    {
      "addSheet": {
        "properties": { 'sheetType': 'GRID',
          "title": "CashFlow",
          "gridProperties": {
            "rowCount": 60,
            "columnCount": 12
          }
        }
      }
    }
  ]
}).execute()


# List of ID
spreadsheet = service.spreadsheets().get(spreadsheetId = spreadsheetId).execute()
sheetList = spreadsheet.get('sheets')
for sheet in sheetList:
    print(sheet['properties']['sheetId'], sheet['properties']['title'])
    
sheetId = sheetList[0]['properties']['sheetId']

print('Current с Id = ', sheetId)

0 Dirty_data
175054880 CashFlow
Current с Id =  0


In [23]:
ID = sheet['properties']['sheetId']

###  The table publishing

In [24]:
results = service.spreadsheets().values().batchUpdate(spreadsheetId = spreadsheetId, body = {
    "valueInputOption": "USER_ENTERED", # Users mod
    "data": [
        {"range": "Dirty_data!B2:zzz100",
         "majorDimension": "ROWS",     # Row first
         "values": [dirty.columns.tolist()
                   ]}
    ]
}).execute()

In [25]:
final = dirty.to_numpy().tolist()

In [26]:
results = service.spreadsheets().values().batchUpdate(spreadsheetId = spreadsheetId, body = {
    "valueInputOption": "USER_ENTERED", # Users mode
     "data": [
        {"range": "Dirty_data!B3:zzz100",
         "majorDimension": "ROWS",     # Row first
         "values": final}
    ]
}).execute()

In [27]:
result['amount'] = 0 - result['amount']

In [28]:
result.pivot_table(index = ['account_id','category', 'amount'], values = 'date',  aggfunc = 'sum').sort_values(['account_id','date']).reset_index()

,account_id,category,amount,date
0,3rG6aLzoAdSwyMn8lyJMIWnyEoeGx7HqBBk91,Transfer / Credit,4.22,2020-05-10
1,3rG6aLzoAdSwyMn8lyJMIWnyEoeGx7HqBBk91,Payment / Credit Card,-25.00,2020-05-15
2,BMDa8v1Ab9cWle14klxeHzaWM4v63qfwQQd3G,Travel / Taxi,-6.33,2020-04-28
3,BMDa8v1Ab9cWle14klxeHzaWM4v63qfwQQd3G,Food and Drink / Restaurants,-89.40,2020-05-11
4,BMDa8v1Ab9cWle14klxeHzaWM4v63qfwQQd3G,Food and Drink / Restaurants / Coffee Shop,-4.33,2020-05-12
5,BMDa8v1Ab9cWle14klxeHzaWM4v63qfwQQd3G,Food and Drink / Restaurants / Fast Food,-12.00,2020-05-12
6,BMDa8v1Ab9cWle14klxeHzaWM4v63qfwQQd3G,Travel / Airlines and Aviation Services,500.00,2020-05-13
7,BMDa8v1Ab9cWle14klxeHzaWM4v63qfwQQd3G,Travel / Taxi,-5.40,2020-05-15
8,a8BwAP4Kb9hQyMo5By3MHLNaWv4xGVt7zz8gE,Transfer / Debit,-5850.00,2020-05-14
9,dkDqKNjB68tGDQbvmDaQHKgpP6Z4AXcZmmv4k,Travel / Airlines and Aviation Services,-500.00,2020-04-30


### Add Kind

In [29]:
def flow(i):
    if i < 0:
        return 'expenses'
    else:
        return 'income'

In [30]:
result['kind'] = result['amount'].apply(flow)

In [31]:
result.head()

,name,amount,category,date,account_id,available,current,limit,kind
0,CD DEPOSIT .INITIAL.,-1000.0,Transfer / Deposit,2020-05-14,xGgpoR9yn5Uo3K5m93pKTlXpDEPkBACnqqxDo,0.0,1000.0,0.0,expenses
1,Tectra Inc,-500.0,Food and Drink / Restaurants,2020-05-25,dkDqKNjB68tGDQbvmDaQHKgpP6Z4AXcZmmv4k,0.0,410.0,2000.0,expenses
2,AUTOMATIC PAYMENT - THANK,-2078.5,Payment,2020-05-24,dkDqKNjB68tGDQbvmDaQHKgpP6Z4AXcZmmv4k,0.0,410.0,2000.0,expenses
3,KFC,-500.0,Food and Drink / Restaurants / Fast Food,2020-05-24,dkDqKNjB68tGDQbvmDaQHKgpP6Z4AXcZmmv4k,0.0,410.0,2000.0,expenses
4,Madison Bicycle Shop,-500.0,Shops / Sporting Goods,2020-05-24,dkDqKNjB68tGDQbvmDaQHKgpP6Z4AXcZmmv4k,0.0,410.0,2000.0,expenses


### The scores spliting 

In [32]:
def part(i):
    a = result[result['account_id'] == i].pivot_table(index = ['account_id','category', 'name','amount', 'kind'], 
                                                      values = 'date', 
                                                      aggfunc = 'sum').sort_values(['account_id','date']).reset_index()
    return a
    

In [33]:
ONE = part('3rG6aLzoAdSwyMn8lyJMIWnyEoeGx7HqBBk91')

In [34]:
TWO = part('BMDa8v1Ab9cWle14klxeHzaWM4v63qfwQQd3G')

In [35]:
THREE = part('a8BwAP4Kb9hQyMo5By3MHLNaWv4xGVt7zz8gE')

In [36]:
FOUR = part('dkDqKNjB68tGDQbvmDaQHKgpP6Z4AXcZmmv4k')

In [37]:
FIVE = part('xGgpoR9yn5Uo3K5m93pKTlXpDEPkBACnqqxDo')

### Sum for calc zero income

In [38]:
def plus(NOM):
    b = abs(NOM['amount']).sum()
    a = NOM.groupby('kind')['amount'].sum()[0]
    return round(b+a,2)  

In [39]:
results = service.spreadsheets().values().batchUpdate(spreadsheetId = spreadsheetId, body = {
    "valueInputOption": "USER_ENTERED", # Users mode input
    "data": [
        {"range": "CashFlow!B2:zzz100",
         "majorDimension": "ROWS",     
         "values": [ONE.columns.tolist()]
                   },
        
        {"range": "CashFlow!B3:zzz100",
         "majorDimension": "ROWS",     
         "values": ONE.to_numpy().tolist()
                   },
        {"range": "CashFlow!B5:zzz100",
         "majorDimension": "ROWS",     
         "values": [[" ", " ","Total", ONE['amount'].sum(), " "]]
                   },
           {"range": "CashFlow!B6:zzz100",
         "majorDimension": "ROWS",     
         "values": [[" ", " ","Expenses", ONE.groupby('kind')['amount'].sum()[0], " "]]
                   },
          {"range": "CashFlow!B7:zzz100",
         "majorDimension": "ROWS",     
         "values": [[" ", " ","Income", plus(ONE), " "]]
                   },
     
        
        {"range": "CashFlow!B8:zzz100",
         "majorDimension": "ROWS",     
         "values": TWO.to_numpy().tolist()
                   },
        {"range": "CashFlow!B14:zzz100",
         "majorDimension": "ROWS",     
         "values": [[" "," ", "Total", TWO['amount'].sum(), " "]]
                   },
        {"range": "CashFlow!B15:zzz100",
         "majorDimension": "ROWS",     
         "values": [[" ", " ","Expenses", TWO.groupby('kind')['amount'].sum()[0], " "]]
                   },
          {"range": "CashFlow!B16:zzz100",
         "majorDimension": "ROWS",     
         "values": [[" ", " ","Income", plus(TWO), " "]]
                   },
        
        {"range": "CashFlow!B17:zzz100",
         "majorDimension": "ROWS",     
         "values": THREE.to_numpy().tolist()
                   },
        {"range": "CashFlow!B18:zzz100",
         "majorDimension": "ROWS",     
         "values": [[" "," ", "Total", THREE['amount'].sum(), " "]]
                   },
          {"range": "CashFlow!B19:zzz100",
         "majorDimension": "ROWS",     
         "values": [[" ", " ","Expenses", THREE.groupby('kind')['amount'].sum()[0], " "]]
                   },
          {"range": "CashFlow!B20:zzz100",
         "majorDimension": "ROWS",     
         "values": [[" ", " ","Income",plus(THREE), " "]]
                   },
        
         {"range": "CashFlow!B17:zzz100",
         "majorDimension": "ROWS",     
         "values": FOUR.to_numpy().tolist()
                   },
        {"range": "CashFlow!B23:zzz100",
         "majorDimension": "ROWS",     
         "values": [[" "," ", "Total", FOUR['amount'].sum(), " "]]
                   },
          {"range": "CashFlow!B24:zzz100",
         "majorDimension": "ROWS",     
         "values": [[" ", " ","Expenses", FOUR.groupby('kind')['amount'].sum()[0], " "]]
                   },
          {"range": "CashFlow!B25:zzz100",
         "majorDimension": "ROWS",     
         "values": [[" ", " ","Income", plus(FOUR), " "]]
                   },
        
        
         {"range": "CashFlow!B26:zzz100",
         "majorDimension": "ROWS",     
         "values": FIVE.to_numpy().tolist()
                   },
        {"range": "CashFlow!B27:zzz100",
         "majorDimension": "ROWS",     
         "values": [[" "," ", "Total", FIVE['amount'].sum(), " "]]
                   },
          {"range": "CashFlow!B28:zzz100",
         "majorDimension": "ROWS",     
         "values": [[" ", " ","Expenses", FIVE.groupby('kind')['amount'].sum()[0], " "]]
                   },
          {"range": "CashFlow!B29:zzz100",
         "majorDimension": "ROWS",     
         "values": [[" ", " ","Income", plus(FIVE), " "]]
                   },
        
        
         {"range": "CashFlow!B30:zzz100",
         "majorDimension": "ROWS",     
         "values": [[" "," ", "Grand Total", result['amount'].sum(), " "]]
                   }
    ]
}).execute()

In [40]:
def merge(startrow, endrow, startcol, endcol):
    results = service.spreadsheets().batchUpdate(
    spreadsheetId = spreadsheetId,
    body = {
        "requests": [
            {'mergeCells': {'range': {'sheetId': ID,
                          'startRowIndex': startrow,
                          'endRowIndex': endrow,
                          'startColumnIndex': startcol,
                          'endColumnIndex': endcol},
                        
                'mergeType': 'MERGE_ALL'}},
           
        ]
    }).execute()
    return results

In [41]:
merge(0, 1, 1,7)
#  Add title
results = service.spreadsheets().values().batchUpdate(spreadsheetId = spreadsheetId, body = {
    "valueInputOption": "USER_ENTERED",
    "data": [
        {"range": "CashFlow!B1",
         "majorDimension": "ROWS", # Row first
         "values": [["CashFlow" ] 
                   ]}
    ]
}).execute()

### Format table

In [42]:
merge(1, 30, 0,1)

{'spreadsheetId': '1cbkFqsdsJbmB6CPMgCJ-7q8Qz_JnvoNdyR8x3uINjt0',
 'replies': [{}]}

In [43]:
merge(2, 4, 1,2)

{'spreadsheetId': '1cbkFqsdsJbmB6CPMgCJ-7q8Qz_JnvoNdyR8x3uINjt0',
 'replies': [{}]}

In [44]:
merge(7, 13,1,2)

{'spreadsheetId': '1cbkFqsdsJbmB6CPMgCJ-7q8Qz_JnvoNdyR8x3uINjt0',
 'replies': [{}]}

In [45]:
merge(16, 22,1,2)

{'spreadsheetId': '1cbkFqsdsJbmB6CPMgCJ-7q8Qz_JnvoNdyR8x3uINjt0',
 'replies': [{}]}

In [46]:
#Format columns wide
results = service.spreadsheets().batchUpdate(spreadsheetId = spreadsheetId, body = {
  "requests": [

    # Columns wide
    {
      "updateDimensionProperties": {
        "range": {
          "sheetId": ID,
          "dimension": "COLUMNS",  
          "startIndex": 0, 
          "endIndex": 1 
        },
        "properties": {
          "pixelSize": 20 
        },
        "fields": "pixelSize" 
      }
    },


    {
      "updateDimensionProperties": {
        "range": {
          "sheetId": ID,
          "dimension": "COLUMNS",
          "startIndex": 1,
          "endIndex": 4
        },
        "properties": {
          "pixelSize": 300
        },
        "fields": "pixelSize"
      }
    },

    
    {
      "updateDimensionProperties": {
        "range": {
          "sheetId": ID,
          "dimension": "COLUMNS",
          "startIndex": 4,
          "endIndex": 5
        },
        "properties": {
          "pixelSize": 100
        },
        "fields": "pixelSize"
      }
    }
  ]
}).execute()

In [47]:
results = service.spreadsheets().batchUpdate(spreadsheetId = spreadsheetId, body = {
  "requests": [

    # Row height
    {
      "updateDimensionProperties": {
        "range": {
          "sheetId": ID,
          "dimension": "ROWS",  
          "startIndex": 0, 
          "endIndex": 2 
        },
        "properties": {
          "pixelSize": 30 
        },
        "fields": "pixelSize" 
      }
    },
       {
      "updateDimensionProperties": {
        "range": {
          "sheetId": ID,
          "dimension": "ROWS",  
          "startIndex": 4, 
          "endIndex": 5 
        },
        "properties": {
          "pixelSize": 30 
        },
        "fields": "pixelSize" 
      }
    },
       {
      "updateDimensionProperties": {
        "range": {
          "sheetId": ID,
          "dimension": "ROWS",  
          "startIndex": 11, 
          "endIndex": 12 
        },
        "properties": {
          "pixelSize": 30 
        },
        "fields": "pixelSize" 
      }
    },
      {
      "updateDimensionProperties": {
        "range": {
          "sheetId": ID,
          "dimension": "ROWS",  
          "startIndex": 13, 
          "endIndex": 14 
        },
        "properties": {
          "pixelSize": 30 
        },
        "fields": "pixelSize" 
      }
    },
      {
      "updateDimensionProperties": {
        "range": {
          "sheetId": ID,
          "dimension": "ROWS",  
          "startIndex": 20, 
          "endIndex": 21 
        },
        "properties": {
          "pixelSize": 30 
        },
        "fields": "pixelSize" 
      }
    },
      {
      "updateDimensionProperties": {
        "range": {
          "sheetId": ID,
          "dimension": "ROWS",  
          "startIndex": 22, 
          "endIndex": 23 
        },
        "properties": {
          "pixelSize": 30 
        },
        "fields": "pixelSize" 
      }
    },
      {
      "updateDimensionProperties": {
        "range": {
          "sheetId": ID,
          "dimension": "ROWS",  
          "startIndex": 23, 
          "endIndex": 24 
        },
        "properties": {
          "pixelSize": 40 
        },
        "fields": "pixelSize" 
      }
    },
   
  ]
}).execute()

In [48]:
def cell(startrow, endrow, startcol, endcol):
    results = service.spreadsheets().batchUpdate(
    spreadsheetId = spreadsheetId,
    body = 
{
  "requests": 
  [
    {
      "repeatCell": 
      {
        "cell": 
        {
          "userEnteredFormat": 
          {
            "horizontalAlignment": 'CENTER',
            "backgroundColor": {
                "red": 0.8,
                "green": 0.8,
                "blue": 0.8,
                "alpha": 1
            },
            "textFormat":
             {
               "bold": True,
               "fontSize": 10
             }
          }
        },
        "range": 
        {
          "sheetId": ID,
          "startRowIndex": startrow,
          "endRowIndex": endrow,
          "startColumnIndex": startcol,
          "endColumnIndex": endcol
        },
        "fields": "userEnteredFormat"
      }
    }
  ]
}).execute()
    return results
    

In [49]:
cell(1, 2, 1, 7)

{'spreadsheetId': '1cbkFqsdsJbmB6CPMgCJ-7q8Qz_JnvoNdyR8x3uINjt0',
 'replies': [{}]}

In [50]:
cell(2, 22, 0, 2)

{'spreadsheetId': '1cbkFqsdsJbmB6CPMgCJ-7q8Qz_JnvoNdyR8x3uINjt0',
 'replies': [{}]}

In [51]:
cell(4, 7, 1, 7)

{'spreadsheetId': '1cbkFqsdsJbmB6CPMgCJ-7q8Qz_JnvoNdyR8x3uINjt0',
 'replies': [{}]}

In [52]:
cell(13, 16, 1, 7)

{'spreadsheetId': '1cbkFqsdsJbmB6CPMgCJ-7q8Qz_JnvoNdyR8x3uINjt0',
 'replies': [{}]}

In [53]:
cell(4, 7, 1, 7)

{'spreadsheetId': '1cbkFqsdsJbmB6CPMgCJ-7q8Qz_JnvoNdyR8x3uINjt0',
 'replies': [{}]}

In [54]:
cell(22, 25, 1, 7)

{'spreadsheetId': '1cbkFqsdsJbmB6CPMgCJ-7q8Qz_JnvoNdyR8x3uINjt0',
 'replies': [{}]}

In [55]:
cell(22, 23, 1, 7)

{'spreadsheetId': '1cbkFqsdsJbmB6CPMgCJ-7q8Qz_JnvoNdyR8x3uINjt0',
 'replies': [{}]}

In [56]:
cell(26, 30, 1, 7)

{'spreadsheetId': '1cbkFqsdsJbmB6CPMgCJ-7q8Qz_JnvoNdyR8x3uINjt0',
 'replies': [{}]}

In [57]:
p=

SyntaxError: invalid syntax (<ipython-input-57-6c2fe2afb67f>, line 1)

In [ ]:
# CELL format TITLE
results = service.spreadsheets().batchUpdate(
    spreadsheetId = spreadsheetId,
    body = 
{
  "requests": 
  [
    {
      "repeatCell": 
      {
        "cell": 
        {
          "userEnteredFormat": 
          {
            "horizontalAlignment": 'CENTER',
            "backgroundColor": {
                "red": 0.8,
                "green": 0.8,
                "blue": 0.8,
                "alpha": 1
            },
            "textFormat":
             {
               "bold": True,
               "fontSize": 14
             }
          }
        },
        "range": 
        {
          "sheetId": ID,
          "startRowIndex": 1,
          "endRowIndex": 2,
          "startColumnIndex": 1,
          "endColumnIndex": 7
        },
        "fields": "userEnteredFormat"
      }
    }
  ]
}).execute()

In [ ]:
#CELL format ID TOP
results = service.spreadsheets().batchUpdate(
    spreadsheetId = spreadsheetId,
    body = 
{
  "requests": 
  [
    {
      "repeatCell": 
      {
        "cell": 
        {
          "userEnteredFormat": 
          {
            "horizontalAlignment": 'CENTER', "verticalAlignment": 'TOP',
              
            "backgroundColor": {
                "red": 1,
                "green": 1,
                "blue": 1,
                "alpha": 1
            },
            "textFormat":
             {
               "bold": False,
               "fontSize": 10
             },
                  
          }
        },
        "range": 
        {
          "sheetId": ID,
          "startRowIndex": 2,
          "endRowIndex": 22,
          "startColumnIndex": 0,
          "endColumnIndex": 2
        },
        "fields": "userEnteredFormat"
      }
    }
  ]
}).execute()

In [ ]:
# CELL format ONE 
results = service.spreadsheets().batchUpdate(
    spreadsheetId = spreadsheetId,
    body = 
{
  "requests": 
  [
    {
      "repeatCell": 
      {
        "cell": 
        {
          "userEnteredFormat": 
          {
            "horizontalAlignment": 'CENTER',
            "backgroundColor": {
                "red": 0.8,
                "green": 0.8,
                "blue": 0.8,
                "alpha": 1
            },
            "textFormat":
             {
               "bold": True,
               "fontSize": 10
             }
          }
        },
        "range": 
        {
          "sheetId": ID,
          "startRowIndex": 4,
          "endRowIndex": 7,
          "startColumnIndex": 1,
          "endColumnIndex": 7
        },
           
        "fields": "userEnteredFormat"
      }
    }
  ]
}).execute()

In [ ]:
# CELL format TWO
results = service.spreadsheets().batchUpdate(
    spreadsheetId = spreadsheetId,
    body = 
{
  "requests": 
  [
    {
      "repeatCell": 
      {
        "cell": 
        {
          "userEnteredFormat": 
          {
            "horizontalAlignment": 'CENTER',
            "backgroundColor": {
                "red": 0.8,
                "green": 0.8,
                "blue": 0.8,
                "alpha": 1
            },
            "textFormat":
             {
               "bold": True,
               "fontSize": 10
             }
          }
        },
        "range": 
         {
          "sheetId": ID,
          "startRowIndex": 13,
          "endRowIndex": 16,
          "startColumnIndex": 1,
          "endColumnIndex": 7
        },
           
        "fields": "userEnteredFormat"
      }
    }
  ]
}).execute()

In [ ]:
# CELL format FOUR
results = service.spreadsheets().batchUpdate(
    spreadsheetId = spreadsheetId,
    body = 
{
  "requests": 
  [
    {
      "repeatCell": 
      {
        "cell": 
        {
          "userEnteredFormat": 
          {
            "horizontalAlignment": 'CENTER',
            "backgroundColor": {
                "red": 0.8,
                "green": 0.8,
                "blue": 0.8,
                "alpha": 1
            },
            "textFormat":
             {
               "bold": True,
               "fontSize": 10
             }
          }
        },
        "range": 
         {
          "sheetId": ID,
          "startRowIndex": 22,
          "endRowIndex": 25,
          "startColumnIndex": 1,
          "endColumnIndex": 7
        },
           
        "fields": "userEnteredFormat"
      }
    }
  ]
}).execute()

In [ ]:
# CELL format
results = service.spreadsheets().batchUpdate(
    spreadsheetId = spreadsheetId,
    body = 
{
  "requests": 
  [
    {
      "repeatCell": 
      {
        "cell": 
        {
          "userEnteredFormat": 
          {
            "horizontalAlignment": 'CENTER',
            "backgroundColor": {
                "red": 0.8,
                "green": 0.8,
                "blue": 0.8,
                "alpha": 1
            },
            "textFormat":
             {
               "bold": True,
               "fontSize": 10
             }
          }
        },
        "range": 
         {
          "sheetId": ID,
          "startRowIndex": 22,
          "endRowIndex": 23,
          "startColumnIndex": 1,
          "endColumnIndex": 7
        },
           
        "fields": "userEnteredFormat"
      }
    }
  ]
}).execute()

In [ ]:
# CELL format
results = service.spreadsheets().batchUpdate(
    spreadsheetId = spreadsheetId,
    body = 
{
  "requests": 
  [
    {
      "repeatCell": 
      {
        "cell": 
        {
          "userEnteredFormat": 
          {
            "horizontalAlignment": 'CENTER',
            "backgroundColor": {
                "red": 0.8,
                "green": 0.8,
                "blue": 0.8,
                "alpha": 1
            },
            "textFormat":
             {
               "bold": True,
               "fontSize": 10
             }
          }
        },
        "range": 
         {
          "sheetId": ID,
          "startRowIndex": 26,
          "endRowIndex": 30,
          "startColumnIndex": 1,
          "endColumnIndex": 7
        },
           
        "fields": "userEnteredFormat"
      }
    }
  ]
}).execute()